In [ ]:
import time
import logging
from logging.config import dictConfig
import numpy as np
import random
from random import randint
from math import ceil, floor
import matplotlib.pyplot as plt
from skimage import io, morphology, exposure
from seed import *
import sys


if __name__ == '__main__':

    logging.basicConfig(filename="runTime.log", level=logging.INFO)

    filenames = ["T1.gif","T2.gif","T3.gif","T4.gif","T5.gif",]
    output_size = 200
    for file in filenames:
        for windowSize in [5,9,11]:     # running for the required window sizes
            start = time.time()         # recording the running time
            textureSynthesis("textures/"+file, windowSize, output_size)
            end = time.time()
            logging.info("\t"+file+"-"+str(windowSize)+"\t:-  "+str(end-start)+" secs")

In [ ]:
def findMatches(temp,convPatches, mask_final, mask_gaussian, windowSize, halfWindow, ErrThreshold):

    temp = np.reshape(temp, windowSize*windowSize)
    mask_gaussian = np.reshape(mask_gaussian, windowSize*windowSize)
    mask_final = np.reshape(mask_final, windowSize*windowSize)
    total_weight = np.sum(np.multiply(mask_gaussian, mask_final))
    distance = (convPatches-temp)**2
    ssd = np.sum((distance*mask_gaussian*mask_final) / total_weight, axis=1)
    min_error = min(ssd)
    j = int(((2 * halfWindow + 1) ** 2) / 2)
    return [[err, convPatches[i][j]] for i, err in enumerate(ssd) if err <= min_error*(1+ErrThreshold)]

In [ ]:
def seed_of_image(img, seed_size, output_rows, output_cols):

    (rows,cols) = np.shape(img)  #obtain dimensions of image

    row_rand = randint(0, rows - seed_size)
    col_rand = randint(0, cols - seed_size)

    seed = img[row_rand:row_rand + seed_size, col_rand:col_rand + seed_size]

    synthesized_image = np.zeros((output_rows,output_cols))

    row_center = floor(output_rows / 2)
    col_center = floor(output_cols / 2)
    half_seedSize = floor(seed_size/2)
    synthesized_image[row_center - half_seedSize : row_center + half_seedSize+1, col_center - half_seedSize : col_center + half_seedSize+1] = seed

    map_filled = np.zeros((output_rows,output_cols),dtype=np.int8)
    map_filled[row_center - half_seedSize : row_center + half_seedSize+1, col_center - half_seedSize : col_center + half_seedSize+1] = np.ones((seed_size, seed_size))

    return synthesized_image,map_filled

In [ ]:
def textureSynthesis(imageFile, windowSize, output_size):

    print(imageFile.split('.')[0]+"-"+str(windowSize)+"-synth.gif")
    img = io.imread(imageFile)
    img = img/255.0
    row, col = np.shape(img)
    error_threshold = 0.1
    max_error_threshold = 0.3
    sigma = windowSize/6.4
    seed = 3
    halfWindow = (windowSize - 1) // 2
    totalPixels = output_size*output_size
    filledPixels = seed * seed

    synthesized_image, map_filled = seed_of_image(img, seed, output_size, output_size)

    convPatches = convolutionPatches(img,halfWindow)

    synthImagePad = np.lib.pad(synthesized_image, halfWindow, mode='constant', constant_values=0)
    map_filledPad = np.lib.pad(map_filled, halfWindow, mode='constant', constant_values=0)

    gaussMask = GaussMask(windowSize,sigma)
    
    while filledPixels < totalPixels:

        progress = False

        pixel_list = np.nonzero(morphology.binary_dilation(map_filled) - map_filled)

        neighbors = []
        neighbors.append([np.sum(map_filled[pixel_list[0][i] - halfWindow : pixel_list[0][i] + halfWindow + 1, pixel_list[1][i] - halfWindow : pixel_list[1][i] + halfWindow + 1]) for i in range(len(pixel_list[0]))])
        decreasingOrder = np.argsort(-np.array(neighbors, dtype=int))

        for i in decreasingOrder[0]:

            temp = synthImagePad[pixel_list[0][i] - halfWindow + halfWindow:pixel_list[0][i] + halfWindow + halfWindow + 1, pixel_list[1][i] - halfWindow + halfWindow:pixel_list[1][i] + halfWindow + halfWindow + 1]
            validMask = map_filledPad[pixel_list[0][i] - halfWindow + halfWindow:pixel_list[0][i] + halfWindow + halfWindow + 1, pixel_list[1][i] - halfWindow + halfWindow:pixel_list[1][i] + halfWindow + halfWindow + 1]

            best_match = findMatches(temp,convPatches,validMask,gaussMask,windowSize, halfWindow, ErrThreshold)

            bestMatch = randint(0, len(best_match)-1)

            if best_match[bestMatch][0]<=max_error_threshold:
        
                synthImagePad[halfWindow+pixel_list[0][i]][halfWindow+pixel_list[1][i]] = best_match[bestMatch][1]
                synthesized_image[pixel_list[0][i]][pixel_list[1][i]] = best_match[bestMatch][1]
                map_filledPad[halfWindow+pixel_list[0][i]][halfWindow+pixel_list[1][i]] = 1
                map_filled[pixel_list[0][i]][pixel_list[1][i]] = 1
                filledPixels+=1

                progress = True

        if not progress:
            max_error_threshold *= 1.1
        i = (filledPixels/totalPixels)*100
        sys.stdout.write("\r%d%%" % i)
        sys.stdout.flush()

    io.imsave(imageFile.split('.')[0]+"-"+str(windowSize)+"-synth.gif", synthesized_image)
    plt.show()
    return


def GaussMask(windowSize, sigma):
    x, y = np.mgrid[-windowSize//2 + 1:windowSize//2 + 1, -windowSize//2 + 1:windowSize//2 + 1]
    g = np.exp(-((x**2 + y**2)/(2.0*sigma**2)))
    return g/g.sum()


def convolutionPatches(src_img, halfWindow):
    convPatches = []
    for i in range(halfWindow, src_img.shape[0]-halfWindow-1):
        for j in range(halfWindow, src_img.shape[1]-halfWindow-1):
            convPatches.append(np.reshape(src_img[i-halfWindow:i + halfWindow + 1, j - halfWindow: j + halfWindow + 1], (2 * halfWindow + 1) ** 2))
    convPatches = np.double(convPatches)
    return convPatches